In [ ]:
import pandas as pd  
import numpy as np
import random
import matplotlib.pyplot as plt

floor_time = 0.0042 # Time to move 1 floor (15 second)
getin_time = 0.0028 # Time to get everyone on elevator
kalanlar = 0

floor_count =random.randint(10,20) + 1# For Garage
elevator_count = 2
flat_count = 4
flat_matrix = []
person_matrix= []
day_matrix = []
hour_matrix = []

# Creating the Apartment
def workingdays(floor,flat,person,car):  
    if(np.random.rand()>0.85):
        worktype = random.randint(0,2)
        if(worktype == 0):#Tuesday to Saturday
            day_matrix.append([floor,flat,person,0,1,1,1,1,1,0])
            callhours(floor,1,car)
        elif(worktype == 1):#Thursday to Monday
            day_matrix.append([floor,flat,person,1,0,0,1,1,1,1])
            callhours(floor,2,car)
        else:
            day_matrix.append([floor,flat,person,1,1,0,0,1,1,1])
            callhours(floor,3,car)
    else:
        day_matrix.append([floor,flat,person,1,1,1,1,1,0,0])
        callhours(floor,0,car)

def callhours(floor,daytype,car):
    dayshiftout = 7
    dayshiftin = 17
    midshiftout = 15
    midshiftin = 2
    nightshiftout = 21
    nightshiftin = 6
    destination = (car+1)%2
    hours = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,destination,floor]
    shift = np.random.rand()
    if(shift<0.84):
        shiftout = dayshiftout
        shiftin = dayshiftin
    elif(0.90>shift>0.84):
        shiftout = midshiftout
        shiftin = midshiftin
    else:
        shiftout = nightshiftout
        shiftin = nightshiftin
        
    for days in range(5):
        delay = np.round(random.gauss(0, 0.85))/4
        if(daytype == 0):
            hours[days] = shiftout+delay
            hours[days+7] = shiftin+delay
        elif(daytype == 1):
            hours[days+1] = shiftout+delay
            hours[days+8] = shiftin+delay
        elif(daytype == 2):
            if(days == 5):
                hours[0] = shiftout+delay
                hours[7] = shiftin+delay
            else:
                hours[days+3] = shiftout+delay
                hours[days+9] = shiftin+delay
        else:
            if(days == 4):
                hours[0] = shiftout+delay
                hours[7] = shiftin+delay
            elif(days == 5):
                hours[1] = shiftout+delay
                hours[8] = shiftin+delay
            else:
                hours[days+4] = shiftout+delay
                hours[days+10] = shiftin+delay
        
    #For Off Days
    for weekoff in range(2):
        outhour = -1
        inhour = -1
        if(np.random.rand()<0.30):
            outhour = random.randint(11,15)+(random.randint(0,2)/4)
            inhour = outhour+1+(random.randint(1,4)/4)
        
        if(daytype == 0):
            hours[5+weekoff] = outhour
            hours[12+weekoff] = inhour
        elif(daytype == 1):
            if(weekoff == 0):
                hours[6] = outhour
                hours[13] = inhour
            else:
                hours[0] = outhour
                hours[7] = inhour
        elif(daytype == 2):
            hours[1+weekoff] = outhour
            hours[8+weekoff] = inhour
        else:
            hours[2+weekoff] = outhour
            hours[9+weekoff] = inhour
    hour_matrix.append(hours)
for floor in range(1,floor_count):
    for flat in range(flat_count):
        # Person Count
        person_count = np.round(random.gauss(2.5, 0.56))
        flat_matrix.append([floor,flat,person_count])
        firstgender = 0
        firstage = 0
        for person in range(int(person_count)):
            if(person == 0):
                if(np.random.rand() > 0.50): # Due to TUIK Statistics 0 for men 1 for women
                    gender = 0
                else:
                    gender = 1
                    firstgender = 1
                age = np.round(random.gauss(35, 4.2))
                firstage = age
                #Does this person has car? 
                if(np.random.rand() > 0.35):
                    car = 0
                else:
                    car = 1
                workingdays(floor,flat,person,car)
            elif(person == 1):
                if(np.random.rand() < 0.75):
                    gender = (firstgender+1)%2
                    
                else:
                    gender = firstgender
                age = np.round(random.gauss(firstage, 1.5))
                if(np.random.rand() > 0.35):
                    car = 0
                else:
                    car = 1
                if(np.random.rand()>0.114):
                    workingdays(floor,flat,person,car)
            else:
                if(np.random.rand() > 0.03):
                    car = 0
                    age = abs(np.round(random.gauss(random.randint(0,(firstage-21)), 0.15)))#abs for to not get -0.0
                    if(age > 7):
                        day_matrix.append([floor,flat,person,1,1,1,1,1,0,0])
                        callhours(floor,0,0)
                else:
                    age = np.round(random.gauss(firstage, 1.5))
                    if(np.random.rand() > 0.35):
                        car = 0
                    else:
                        car = 1
                    if(np.random.rand()>0.114):
                        workingdays(floor,flat,person,car)
                if(np.random.rand() > 0.58):
                    gender = 0
                else:
                    gender = 1
            person_matrix.append([floor,flat,person,gender,age,car])
print("Floor on the building:",floor_count,"\nPeople living in the building:",len(person_matrix))
# Creating the Apartment

flat_matrix = np.asarray(flat_matrix)
person_matrix = np.asarray(person_matrix)
day_matrix = np.asarray(day_matrix)
hour_matrix = np.asarray(hour_matrix)

#Optimization

def elevator_controller(floor_1,floor_2,request_matrix,time,kalanlar):
    floor_time = 0.0042 # Time to move 1 floor (15 second)
    getin_time = 0.0028 # Time to get everyone on elevator
    current_floor1 = floor_1
    current_floor2 = floor_2
    max_capacity = 16
    elevator1_capacity = 0
    elevator2_capacity = 0
    basetime = time
    time1 = 0
    time2 = 0
    waiting_time = 0
    if(isinstance(request_matrix,np.ndarray)):
        time_var = len(request_matrix[:,0])
    else:
        time_var = 0
    if(isinstance(kalanlar, np.ndarray)):
      if(isinstance(request_matrix,np.ndarray)):
        floor_requests = np.column_stack([kalanlar,request_matrix])
        kalanlar = 0
      else:
        floor_requests = kalanlar
        kalanlar = 0
    else:
        floor_requests = request_matrix
    if(isinstance(floor_requests, np.ndarray)):
      destination1 = max(floor_requests[:][0]) 
      destination2 = max(floor_requests[:][0]) 
    else:
      destination1 = 0
      destination2 = 0 
    elevator1_persons = []
    elevator2_persons = []

    def get_in_person(floor,matrix,elevator,time,cap1,cap2,per1,per2):
        elevator1_capacity = cap1
        elevator2_capacity = cap2
        elevator1_persons = per1
        elevator2_persons = per2
        persons = matrix[matrix[matrix[:,0]==floor][:,2]<(basetime+time)]
        others = np.column_stack([matrix[matrix[:,0]!=floor],matrix[matrix[matrix[:,0]==floor][:,2]>basetime]])
        person_count = len(persons)
        for i in range(person_count):
            if(elevator == 1):
                if(elevator1_capacity+1 <= max_capacity):
                    elevator1_persons.append(persons[i,1])
                    elevator1_capacity +=1
                    np.delete(persons,0,0)
            else:
                if(elevator2_capacity+1 <= max_capacity):
                    elevator2_persons.append(persons[i,1])
                    elevator2_capacity +=1
                    np.delete(persons,0,0)
        floor_requests = np.column_stack([persons,others])
        return elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons
    def get_off_person(floor,elevator,cap1,cap2,per1,per2):
        elevator1_capacity = cap1
        elevator2_capacity = cap2
        elevator1_persons = per1
        elevator2_persons = per2
        if(elevator == 1):
            if(len(elevator1_persons)!= 0):
              elevator1_persons = elevator1_persons[elevator1_persons[:] != floor]
              elevator1_capacity = len(elevator1_persons)
        else:
            if(len(elevator2_persons)!= 0):
              elevator2_persons = elevator2_persons[elevator2_persons[:] != floor]
              elevator2_capacity = len(elevator2_persons)
        return elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons
    while(time1<=1 or time2<=1):
        if(elevator1_capacity != max_capacity & elevator2_capacity != max_capacity):
            if(current_floor1 != destination1 & current_floor2 != destination2):
                if(current_floor1 > destination1):
                    getin = get_in_person(current_floor1,floor_requests,1,time1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getin[0]
                    elevator2_capacity = getin[1]
                    elevator1_persons = getin[2]
                    elevator2_persons = getin[3]
                    getoff = get_off_person(current_floor1,1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getoff[0]
                    elevator2_capacity = getoff[1]
                    elevator1_persons = getoff[2]
                    elevator2_persons = getoff[3]
                    waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
                    time1 += getin_time
                    current_floor1 -=1
                else:
                    current_floor1 +=1
                if(current_floor2 > destination2):
                    getin = get_in_person(current_floor2,floor_requests,2,time2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getin[0]
                    elevator2_capacity = getin[1]
                    elevator1_persons = getin[2]
                    elevator2_persons = getin[3]
                    getoff =get_off_person(current_floor2,2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getoff[0]
                    elevator2_capacity = getoff[1]
                    elevator1_persons = getoff[2]
                    elevator2_persons = getoff[3]
                    time2 += getin_time
                    waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
                    current_floor2 -=1
                else:
                    current_floor2 += 1
                time1 += floor_time
                time2 += floor_time
                waiting_time += floor_time*(time_var+elevator1_capacity+elevator2_capacity)
            elif(current_floor1 == destination1 & current_floor2 == destination2):
                if(destination1 == 0 & destination2 == 0):
                    getin = get_in_person(current_floor1,floor_requests,1,time1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getin[0]
                    elevator2_capacity = getin[1]
                    elevator1_persons = getin[2]
                    elevator2_persons = getin[3]
                    getoff = get_off_person(current_floor1,1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getoff[0]
                    elevator2_capacity = getoff[1]
                    elevator1_persons = getoff[2]
                    elevator2_persons = getoff[3]
                    time1 += getin_time
                    getin = get_in_person(current_floor2,floor_requests,2,time2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getin[0]
                    elevator2_capacity = getin[1]
                    elevator1_persons = getin[2]
                    elevator2_persons = getin[3]
                    getoff = get_off_person(current_floor2,2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getoff[0]
                    elevator2_capacity = getoff[1]
                    elevator1_persons = getoff[2]
                    elevator2_persons = getoff[3]
                    time2 += getin_time
                    waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
                    elevator1_capacity,elevator2_capacity = 0
                    destination1,destination2 = max(floor_requests[:,2])
                elif(destination1 == 0):
                    getin = get_in_person(current_floor1,floor_requests,1,time1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getin[0]
                    elevator2_capacity = getin[1]
                    elevator1_persons = getin[2]
                    elevator2_persons = getin[3]
                    getoff = get_off_person(current_floor1,1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getoff[0]
                    elevator2_capacity = getoff[1]
                    elevator1_persons = getoff[2]
                    elevator2_persons = getoff[3]
                    time1 += getin_time
                    waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
                    destination1 = max(floor_requests[:,2])
                    if(current_floor2 > destination2):
                        current_floor2 -=1
                        getin = get_in_person(current_floor2,floor_requests,2,time2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                        elevator1_capacity = getin[0]
                        elevator2_capacity = getin[1]
                        elevator1_persons = getin[2]
                        elevator2_persons = getin[3]
                        getoff = get_off_person(current_floor2,2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                        elevator1_capacity = getoff[0]
                        elevator2_capacity = getoff[1]
                        elevator1_persons = getoff[2]
                        elevator2_persons = getoff[3]
                        time2 += getin_time
                    else:
                        current_floor2 += 1
                elif(destination2 == 0):
                    getin = get_in_person(current_floor2,floor_requests,2,time2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getin[0]
                    elevator2_capacity = getin[1]
                    elevator1_persons = getin[2]
                    elevator2_persons = getin[3]
                    getoff = get_off_person(current_floor2,2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                    elevator1_capacity = getoff[0]
                    elevator2_capacity = getoff[1]
                    elevator1_persons = getoff[2]
                    elevator2_persons = getoff[3]
                    time2 += getin_time
                    waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
                    destination2 = max(floor_requests[:,2])
                    if(current_floor1 > destination1):
                        current_floor1 -=1
                        getin = get_in_person(current_floor1,floor_requests,1,time1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                        elevator1_capacity = getin[0]
                        elevator2_capacity = getin[1]
                        elevator1_persons = getin[2]
                        elevator2_persons = getin[3]
                        getoff = get_off_person(current_floor1,1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                        elevator1_capacity = getoff[0]
                        elevator2_capacity = getoff[1]
                        elevator1_persons = getoff[2]
                        elevator2_persons = getoff[3]
                        time1 += getin_time
                    else:
                        current_floor1 +=1
            time1 += floor_time
            time2 += floor_time
            waiting_time += floor_time*(time_var+elevator1_capacity+elevator2_capacity)
        elif(elevator1_capacity == max_capacity):
            getoff = get_off_person(current_floor1,1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
            elevator1_capacity = getoff[0]
            elevator2_capacity = getoff[1]
            elevator1_persons = getoff[2]
            elevator2_persons = getoff[3]
            time1 += getin_time
            waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
            current_floor1 -=1
            if(current_floor2 > destination2):
                current_floor2 -=1
                getin = get_in_person(current_floor2,floor_requests,2,time2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                elevator1_capacity = getin[0]
                elevator2_capacity = getin[1]
                elevator1_persons = getin[2]
                elevator2_persons = getin[3]
                getoff = get_off_person(current_floor2,2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                elevator1_capacity = getoff[0]
                elevator2_capacity = getoff[1]
                elevator1_persons = getoff[2]
                elevator2_persons = getoff[3]
                time2 += getin_time
            else:
                current_floor2 += 1
            time1 += floor_time
            time2 += floor_time
            waiting_time += floor_time*(time_var+elevator1_capacity+elevator2_capacity)
        elif(elevator2_capacity == max_capacity):
            getoff = get_off_person(current_floor2,2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
            elevator1_capacity = getoff[0]
            elevator2_capacity = getoff[1]
            elevator1_persons = getoff[2]
            elevator2_persons = getoff[3]
            time2 += getin_time
            waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
            current_floor2 -=1
            if(current_floor1 > destination1):
                current_floor1 -=1
                getin = get_in_person(current_floor1,floor_requests,1,time1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                elevator1_capacity = getin[0]
                elevator2_capacity = getin[1]
                elevator1_persons = getin[2]
                elevator2_persons = getin[3]
                getoff = get_off_person(current_floor1,1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
                elevator1_capacity = getoff[0]
                elevator2_capacity = getoff[1]
                elevator1_persons = getoff[2]
                elevator2_persons = getoff[3]
                time1 += getin_time
            else:
                current_floor1 +=1
            time1 += floor_time 
            waiting_time += floor_time*(time_var+elevator1_capacity+elevator2_capacity)
            time2 += floor_time   
        else:
            getoff = get_off_person(current_floor1,1,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
            elevator1_capacity = getoff[0]
            elevator2_capacity = getoff[1]
            elevator1_persons = getoff[2]
            elevator2_persons = getoff[3]
            getoff = get_off_person(current_floor2,2,elevator1_capacity,elevator2_capacity,elevator1_persons,elevator2_persons)
            elevator1_capacity = getoff[0]
            elevator2_capacity = getoff[1]
            elevator1_persons = getoff[2]
            elevator2_persons = getoff[3]
            time1 += getin_time
            time2 += getin_time
            waiting_time += getin_time*(time_var+elevator1_capacity+elevator2_capacity)
            current_floor1 -=1
            current_floor2 -=1
            time1 += floor_time
            time2 += floor_time
            waiting_time += floor_time*(time_var+elevator1_capacity+elevator2_capacity)
    if(len(floor_requests[0]) != 0):
        kalanlar = floor_requests
    return waiting_time,kalanlar



#Optimization
bestsolutions = []
for day in range(7):
    for hour in range(24):
        out_filtre = hour_matrix[hour_matrix[:,day]>=hour]
        out_filtered_matrix = out_filtre[out_filtre[:,day]<(hour+1)]
        out_requests = np.empty((3,len(out_filtered_matrix)))
        out_requests[0] = out_filtered_matrix[:,-1]
        out_requests[1] = out_filtered_matrix[:,-2]
        out_requests[2] = out_filtered_matrix[:,day]
        in_filtre = hour_matrix[hour_matrix[:,day+7]>=hour]
        in_filtered_matrix = in_filtre[in_filtre[:,day+7]<(hour+1)]
        in_requests = np.empty((3,len(in_filtered_matrix)))
        in_requests[0] = in_filtered_matrix[:,-2]
        in_requests[1] = in_filtered_matrix[:,-1]
        in_requests[2] = in_filtered_matrix[:,day+7]
        if(len(in_requests[2])>0 and len(out_requests[2])>0):
            requests = np.column_stack([in_requests,out_requests])
        elif(len(in_requests[2])>0):
            requests = in_requests
        elif(len(out_requests[2])>0):
            requests = out_requests
        else:
            requests = 0
        request_matrix = requests
        solutions = np.empty((4,3))
        for iteration in range(20):
            if(iteration == 0):
                for k in range(4):
                    elevator1 = random.randint(1,floor_count)
                    elevator2 = random.randint(1,floor_count)
                    waitingtime = elevator_controller(elevator1,elevator2,request_matrix,hour,kalanlar)[0]
                    kalanlar = elevator_controller(elevator1,elevator2,request_matrix,hour,kalanlar)[1]
                    np.append(solutions,[elevator1,elevator2,waitingtime])
            else:
                new_solutions = np.empty((4,3))
                ind = np.argmin(solutions[:,2])
                np.append(new_solutions,[solutions[ind,0],solutions[ind,1]])
                np.delete(solutions,ind,0)
                ind = np.argmin(solutions[:,2])
                np.append(new_solutions,[solutions[ind,0],solutions[ind,1]])
                solutions = np.empty((4,3))
                iterations = np.zeros((2,2))
                for j in range(2):
                    if(np.random.rand()>0.40):
                        iterations[j,0] = new_solutions[j,0]-1
                    elif(np.random.rand()>0.40):
                        iterations[j,0] = new_solutions[j,0]+1
                    else:
                        iterations[j,0] = new_solutions[j,0]
                    if(np.random.rand()>0.40):
                        iterations[j,1] = new_solutions[j,0]-1
                    elif(np.random.rand()>0.40):
                        iterations[j,1] = new_solutions[j,0]+1
                    else:
                        iterations[j,1] = new_solutions[j,0]
                np.append(new_solutions,iterations)
                for z in range(4):
                    elevator1 = new_solutions[z,0]
                    elevator2 = new_solutions[z,1]
                    waitingtime = elevator_controller(elevator1,elevator2,request_matrix,hour,kalanlar)[0]
                    kalanlar = elevator_controller(elevator1,elevator2,request_matrix,hour,kalanlar)[1]
            if(iteration == 19):
                index = np.argmin(solutions[:,2])
                bestsolutions.append([solutions[index,0],solutions[index,1],solutions[index,2],hour])
bestsolutions = np.around(np.asarray(bestsolutions), decimals=2)
finalsolutions = np.vsplit(bestsolutions,7)
for final in range(7):
  print("Day:",final+1)
  dataframe = pd.DataFrame(finalsolutions[final],columns=['Elevator 1','Elevator 2','Waiting Time','Hour'])
  print(dataframe)
#Optimization

Floor on the building: 13 
People living in the building: 113
Day: 1
    Elevator 1  Elevator 2  Waiting Time  Hour
0          0.0         1.0           1.0   0.0
1          5.0         1.0           1.0   1.0
2          1.0         1.0           1.0   2.0
3          0.0         1.0           1.0   3.0
4          1.0         1.0           1.0   4.0
5          1.0         0.0           1.0   5.0
6          5.0         1.0           1.0   6.0
7          1.0         1.0           1.0   7.0
8          0.0         0.0           0.0   8.0
9          1.0         1.0           1.0   9.0
10         1.0         1.0           1.0  10.0
11         1.0         1.0           1.0  11.0
12         1.0         1.0           1.0  12.0
13         5.0         1.0           1.0  13.0
14         0.0         0.0           0.0  14.0
15         1.0         0.0           0.0  15.0
16         1.0         1.0           1.0  16.0
17         1.0         0.0           1.0  17.0
18         0.0         0.0           0